In [47]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(["science", "notebook", "grid"])
import itertools

In [63]:
class GasMolecules:
    def __init__(self, boundary:tuple, n_particles:int, v_threshold, radius = float, masses=np.ndarray,
                 dt=float) -> None:
        self.boundary = boundary
        self.v_threshold = v_threshold
        self.n_particles = n_particles
        self.x = np.random.choice([boundary[0], boundary[1]], self.n_particles) + \
                    np.random.uniform(0, 1, self.n_particles) # x position
        self.y = np.random.uniform(boundary[0], boundary[1], self.n_particles) # y position
        self.r = np.array([self.x, self.y]).T # position
        self.color = np.where(self.x > 0, "red", "blue")
        self.radius = np.full(self.n_particles, radius)
        self.v = np.random.uniform(-self.v_threshold, self.v_threshold, (self.n_particles, 2)) # velocity
        self.vx = self.v[:, 0] # x velocity
        self.vy = self.v[:, 1] # y velocity
        self.ids = np.arange(self.n_particles)
        self.masses = masses
        self.t = 0
        self.dt = dt


    def IsColliding(self, r1: np.ndarray, r2:np.ndarray, rd1:float, rd2:float) -> bool:
        """ Check if two circles are colliding  
        Args:
            r1 (np.ndarray): position of the first circle
            r2 (np.ndarray): position of the second circle
            rd1 (float): radius of the first circle
            rd2 (float): radius of the second circle
        Returns:
            bool: True if the circles are colliding"""
        return np.linalg.norm(r1 - r2) < (rd1 + rd2)
    
    def collide(self) -> None:
        """ Change velocity of colliding particles according to elastic collision 
        (momentum conservation and conservation of energy))"""

        # 3d array of all distances for all particle pairs
        r_ij = itertools.combinations(self.r, 2)

        # 2d array of all masses for all particle pairs
        m_ij = np.array(list(itertools.combinations(self.masses, 2))) 

        # 2d array of radii for all particle pairs
        rds_ij = itertools.combinations(self.radius, 2)

        # 2d array of all ids of particle pairs
        ids_ij = itertools.combinations(self.ids, 2)

        # 2d array of all ids of colliding particles
        ids_colliding = []
        for (r1, r2), (rad1, rad2), (id1, id2) in zip(r_ij, rds_ij, ids_ij):
            if self.IsColliding(r1, r2, rad1, rad2):
                ids_colliding.append((id1, id2))
        
        if len(ids_colliding) == 0:
            return None
        ids_colliding = np.array(ids_colliding) # 2d array of all ids of colliding particles

        # Old velocities of colliding particles
        v1_old = self.v[ids_colliding[:, 0]]
        v2_old = self.v[ids_colliding[:, 1]]

        # Masses of colliding particles
        m1 = m_ij[ids_colliding[:, 0]]
        m2 = m_ij[ids_colliding[:, 1]]

        # Position of the colliding particles
        r1 = self.r[ids_colliding[:, 0]]
        r2 = self.r[ids_colliding[:, 1]]

        # Change velocity of colliding particles, '@' is matrix multiplication
        self.v[ids_colliding[:, 0]] -= (2*m2/(m1 + m2)) * \
                                              (np.diag((v1_old - v2_old) @ \
                                                        (r1 - r2).T) / np.sum((r1 - r2)**2, axis=1))\
                                        .reshape(-1, 1) * (r1 - r2)
        self.v[ids_colliding[:, 1]] -= (2*m1/(m1 + m2)) * \
                                              (np.diag((v2_old - v1_old) @ \
                                                        (r2 - r1).T) / np.sum((r1 - r2)**2, axis=1))\
                                        .reshape(-1, 1) * (r2 - r1)
        
    def check_boundary(self) -> None:
        """ Change velocity of particles that hit the boundary """
        self.vx[(self.x >= self.boundary) | (self.x <= -self.boundary)] *= -1 # change x velocity of particles that hit the boundary
        self.vy[(self.y >= self.boundary) | (self.y <= -self.boundary)] *= -1 # change y velocity of particles that hit the boundary

    def update(self) -> None:
        """ Update position and velocity of particles """
        # self.check_boundary() # check if particles hit the boundary
        self.collide() # check if particles collide
        self.r += self.v * self.dt # update position
        self.t += self.dt  # update time

    def __str__(self) -> str:
        return f"""
        Number of particles: {self.n_particles},
        Radius of particles: {np.unique(self.radius)},
        Threshold velocity: {self.v_threshold},
        Time step: {self.dt},
        Masses: {np.unique(self.masses)}
        """
    


In [64]:
# Constants
n_particles = 1000
boundary = (-10, 10)
radius = 0.5
v_threshold = 0.1
masses = np.random.choice([1], n_particles)

# Initialize
gas = GasMolecules(boundary=boundary, n_particles=n_particles, v_threshold=v_threshold, radius=radius, masses=masses, dt=0.01)

# Plot
# plt.scatter(gas.x, gas.y, c=gas.color, s=gas.radius * 6*np.pi)
print(gas)


        Number of particles: 1000,
        Radius of particles: [0.5],
        Threshold velocity: 0.1,
        Time step: 0.01,
        Masses: [1]
        


In [65]:
gas.update()
plt.plot(gas.x, gas.y, "o", c=gas.color, ms=gas.radius * 6*np.pi)